# Exercise 8: Sound transformations

In this exercise you will use the HPS model to creatively transform sounds. There are two parts in this exercise. In the first one you should perform a natural sounding transformation on the speech sound that you used in the previous exercise (E7). In the second part you should select a sound of your choice and do a "creative" transformation. You will have to write a short description of the sound and of the transformation you did, giving the link to the original sound and uploading several transformed sounds.

For this exercise, you can use the `transformations_GUI.py` (in `software/transformations_interface/`) to try things, once decided you can fill up the code in this file. You can also do everything from here and add any new code you wish.

In order to perform a good/interesting transformation you should make sure that you have performed an analysis that is adequate for the type of transformation you want to do. Not every HPS analysis representation will work for every type of sound transformation. There will be things in the analysis that when modified will result in undesired artifacts. In general, for any transformation, it is best to have the harmonic values as smooth and continuous as possible and an stochastic representation as smooth and with as few values as possible. It might be much better to start with an analysis representation that does not result in the best reconstruction in exchange of having smoother and more compact data.

To help you with the exercise, we give a brief description of the transformation parameters used by the HPS transformation function:

1. `freqScaling`: frequency scaling factors to be applied to the harmonics of the sound, in time-value pairs (where value of 1 is no scaling). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The scaling factor is a multiplicative factor, thus a value of 1 is no change. Example: to transpose an octave the sound you can specify `[0, 2, 1, 2]`.
2. `freqStretching`: frequency stretching factors to be applied to the harmonics of the sound, in time-value pairs (value of 1 is no stretching). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The stretching factor is a multiplicative factor whose effect depend on the harmonic number, higher harmonics being more affected that lower ones, thus resulting in an inharmonic effect. A value of 1 results in no transformation. Example: an array like `[0, 1.2, 1, 1.2]` will result in a perceptually large inharmonic effect.
3. `timbrePreservation`: 1 preserves the original timbre, 0 does not. It can only have a value of 0 or of 1. By setting the value to 1 the spectral shape of the original sound is preserved even when the frequencies of the sound are modified. In the case of speech it would correspond to the idea of preserving the identity of the speaker after the transformation.
4. `timeScaling`: time scaling factors to be applied to the whole sound, in time-value pairs (value of 1 is no scaling). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The time scaling factor is a multiplicative factor, thus 1 is no change. Example: to stretch the original sound to twice the original duration, we can specify `[0, 0, 1, 2]`.

All the transformation values can have as many points as desired, but they have to be in the form of an array with time-value pairs, so of even size. For example a good array for a frequency stretching of a sound that has a duration of 3.146 seconds could be: `[0, 1.2, 2.01, 1.2, 2.679, 0.7, 3.146, 0.7]`.

## Part 1. Perform natural sounding transformations of a speech sound

Use the HPS model with the sound `speech-female.wav`, available in the sounds directory, to first analyze and then obtain a natural sounding transformation of the sound. The synthesized sound should sound as different as possible to the original sound while sounding natural. By natural we mean that it should sound like speech, that it could have been possible to be produced by a human, and by listening we should consider it as a speech sound, even though we might not be able to understand it. You should first make sure that you start from a good analysis, then you can do time and/or frequency scaling transformations. The transformation should be done with a single pass, no mixing of sounds coming from different transformations. Since you used the same sound in A7, use that experience to get a good analysis, but consider that the analysis, given that we now want to use it for applying a very strong transformation, might be done differently than what you did in A7.

Write a short paragraph for every transformation, explaining what you wanted to obtain and explaining the transformations you did, giving both the analysis and transformation parameter values (sufficiently detailed for the evaluator to be able to reproduce the analysis and transformation).

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import get_window
import sys, os
sys.path.append('../software/models/')
sys.path.append('../software/transformations/')
import utilFunctions as UF
import stft as STFT
import hpsModel as HPS
import hpsTransformations as HPST
import harmonicTransformations as HT
import IPython.display as ipd

In [18]:
# 1.1 perform an analysis/synthesis using the HPS model

input_file = '../sounds/speech-female.wav'

### set the parameters
window ='hamming'
M = 1357
N = 4096
t = -100
minSineDur = 0.03
nH = 100 
minf0 = 130
maxf0 = 250
f0et = 7
harmDevSlope = 0.01
stocf = 0.1

# no need to modify anything after this
Ns = 512
H = 128

(fs, x) = UF.wavread(input_file)
w = get_window(window, M, fftbins=True)
hfreq, hmag, hphase, stocEnv = HPS.hpsModelAnal(x, fs, w, N, H, t, nH, minf0, maxf0, f0et, harmDevSlope, minSineDur, Ns, stocf)
y, yh, yst = HPS.hpsModelSynth(hfreq, hmag, hphase, stocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=x, rate=fs))
ipd.display(ipd.Audio(data=y, rate=fs))

In [24]:
# 1.2 Perform a transformation from the previous analysis

### define the transformations
freqScaling = np.array([0, 0.8, 1, 0.8 ])
freqStretching = np.array([0, 1, 1, 1])
timbrePreservation = 0
timeScaling = np.array([0, 0, 0.08, 0.2, 1, 1.2])


# no need to modify the following code 
Ns = 512
H = 128

# frequency scaling of the harmonics 
hfreqt, hmagt = HT.harmonicFreqScaling(hfreq, hmag, freqScaling, freqStretching, timbrePreservation, fs)

# time scaling the sound
yhfreq, yhmag, ystocEnv = HPST.hpsTimeScale(hfreqt, hmagt, stocEnv, timeScaling)

# synthesis from the trasformed hps representation 
y, yh, yst = HPS.hpsModelSynth(yhfreq, yhmag, np.array([]), ystocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=y, rate=fs))

### Explain Part 1
Goal: Transform the original female voice into a male voice, and stretch the first syllable to make it sounds hesitant.

Analysis: The fundamental frequencies of the speech range from 130Hz to 250Hz. For speech, we choose the hamming window. So the window size should be equal to or larger than 4*44100/130=1357. To obtain smooth harmonics, we set a larger FFT size N=4096. And to have a smooth stochastic representation, we set a small stocf=0.1

Transformation: Generally, male voice has lower frequency than female. So for freqScaling, we set 0.8 to make it a little lower but still sounds natural. For freqStretching, we change nothing or it will introduce inharmonic effect that a human voice doesn't have. Besides, we set timbrePreservation=0 because we don't want the original female timbre. Lastly, we set the timeScaling [0, 0, 0.08, 0.2, 1, 1.2], in which the 0.08 is where the first syllable ends, and we stretch it to 0.2; the last 1.2 slows down the speech also for the "hesitant" goal.

## Part 2. Perform creative transformations with a sound of your choice

Pick any natural and harmonic sound from Freesound and use the HPS model to do the most creative and interesting transformation you can come up with. Sounding as different as possible from the original sound.

It is essential that you start with a natural harmonic sound. Examples include (but not limited to) any acoustic harmonic instrument, speech, harmonic sound from nature, etc. As long as they have a harmonic structure, you can use it. You can even reuse the sound you used in A7-Part2 or upload your own sound to freesound and then use it.

The sound from Freesound to use could be in any format, but to use the sms-tools software you will have to first convert it to be a monophonic file (one channel), sampling rate of 44100, and 16bits samples.

You can do any interesting transformation with a single pass. It is not allowed to mix sounds obtained from different transformations. The transformed sound need not sound natural. So, time to show some creativity!

Write a short paragraph for every transformation, explaining what you wanted to obtain and explaining the transformations you did, giving both the analysis and transformation parameter values (sufficiently detailed for the evaluator to be able to reproduce the analysis and transformation).

In [32]:
# 1.1 perform an analysis/synthesis using the HPS model
# sound link: https://freesound.org/people/fredgalist/sounds/32235/
### set the parameters
input_file = '../sounds/altosax.wav'
window ='blackman'
M = 945
N = 2048
t = -100
minSineDur = 0.1
nH = 70
minf0 = 280
maxf0 = 700
f0et = 5
harmDevSlope = 0.01
stocf = 0.1

# no need to modify anything after this
Ns = 512
H = 128

(fs, x) = UF.wavread(input_file)
w = get_window(window, M, fftbins=True)
hfreq, hmag, hphase, stocEnv = HPS.hpsModelAnal(x, fs, w, N, H, t, nH, minf0, maxf0, f0et, harmDevSlope, minSineDur, Ns, stocf)
y, yh, yst = HPS.hpsModelSynth(hfreq, hmag, hphase, stocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=x, rate=fs))
ipd.display(ipd.Audio(data=y, rate=fs))

In [31]:
# 1.2 Perform a transformation from the previous analysis

### define the transformations
freqScaling = np.array([0, 1, 
                        0.16, 1.05, 0.18, 0.95, 0.20, 1.05, 0.22, 0.95, 
                        0.24, 1.05, 0.25, 0.95, 0.26, 1.05, 0.27, 0.95, 0.28, 1.05, 0.29, 1.05, 0.30, 1, 
                        2.0, 1])
freqStretching = np.array([0, 1, 1.35, 1, 1.6, 2, 2.0, 1])
timbrePreservation = 1
timeScaling = np.array([0, 0, 0.3, 2.0, 1.35, 3.05, 2.0, 4.5])


# no need to modify anything after this
Ns = 512
H = 128

# frequency scaling of the harmonics 
hfreqt, hmagt = HT.harmonicFreqScaling(hfreq, hmag, freqScaling, freqStretching, timbrePreservation, fs)

# time scaling the sound
yhfreq, yhmag, ystocEnv = HPST.hpsTimeScale(hfreqt, hmagt, stocEnv, timeScaling)

# synthesis from the trasformed hps representation 
y, yh, yst = HPS.hpsModelSynth(yhfreq, yhmag, np.array([]), ystocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=y, rate=fs))

### Explain Part 2
Goal: using frequency scaling to realize a vibrato effect; using time scaling to stretch this vibrato; using frequency stretching to obtain a funny ending.

Analysis: The fundamental frequencies of this alto sax phrase ranges from 280Hz to 700Hz. Since it's a musical signal, we choose blackman window, and set window size (M>=6*44100/280) to be 945. And to obtain smooth harmonics and stochastic representation, we set a lager FFT size N=2048 and small stocf=0.1

Transformation: To realize the vabrato effect, we apply freqScaling on the first note to increase (by 1.05) and decrease (by 0.95) the frequency alternately. We also gradually speed up the vibrato by reduce the time interval. Meanwhile, the first two pairs of timeScaling ([0, 0, 0.3, 2.0]) stretches this vibrato to make it sound better. On the other hand, we apply the freqStretching on the last two note to give it an interesting up and down effect (through this two pairs, [1.6, 2, 2.0, 1]). Moreover, the last two pairs of timeScaling, ([1.35, 3.05, 2.0, 4.5]), stretches the last two note to make the frequency strething sound smoother. Lastly, we want to preserve the orignal sax timbre so we set timbrePreservation=1. 